### 將Excel按照一定條件做切割/篩選

following content first written in 2021.Dec.<br>

 - 說明:
 1. 這邊所輸入的資料可以直接是從站台下載下來的文章列表
 2. 只要用以：(1)快速處理資料檔做一些處理；(2)協助做多重條件輸出，快速撈出符合特定條件的資料

#### 前置作業

In [ ]:
#設定環境
#-*-coding:UTF-8-*- 

import os, time, glob, socket

print("【日期時間】{}".format(time.strftime("%Y/%m/%d %H:%M:%S")))
print("【工作目錄】{}".format(os.getcwd()))
print("【主機名稱】{} ({})".format(socket.gethostname(),socket.gethostbyname(socket.gethostname())))

#設定自動存檔時間
%autosave 60

In [ ]:
#安裝套件

!pip install tqdm
import pandas as pd
from tqdm.notebook import trange, tqdm #可以出現進度條的模組
from time import sleep
import pandas as pd

In [ ]:
#匯入excel檔
##要自行更換路徑

import warnings

with warnings.catch_warnings(record=True):
    warnings.simplefilter("always")
    df1 = pd.read_excel(r"C:\Users\xphoe\OneDrive\文件\Task\eLand\【Unilever】火鍋專案分析報告\維度資料\標記_2021aug.xlsx", 
                       engine="openpyxl", 
                       index_col='監測主題')

with warnings.catch_warnings(record=True):
    warnings.simplefilter("always")
    df2 = pd.read_excel(r"C:\Users\xphoe\OneDrive\文件\Task\eLand\【Unilever】火鍋專案分析報告\維度資料\標記_2021sep.xlsx", 
                       engine="openpyxl", 
                       index_col='監測主題')

with warnings.catch_warnings(record=True):
    warnings.simplefilter("always")
    df3 = pd.read_excel(r"C:\Users\xphoe\OneDrive\文件\Task\eLand\【Unilever】火鍋專案分析報告\維度資料\標記_2021oct.xlsx", 
                       engine="openpyxl", 
                       index_col='監測主題')

In [ ]:
df = pd.concat([df1,df2,df3])

#### 1、按條件將既有欄位做切割

In [ ]:
#將 "來源網站" 的內容，切割、另建立成 "網站" 、 "頻道" 2個不同的欄位
df[['網站', '頻道']] = df.來源網站.str.split(' > ', expand = True)

#將 "發佈時間" 的內容，切割、另建立成 "日期" 、 "時間" 2個不同的欄位
df[['日期', '時間']] = df.發佈時間.str.split(' ', expand = True)

#將 "發佈時間" 的內容，切割、另建立 "年份" 、 "月" 、 "日" 3個不同的欄位
df[['年份', '月', '日']] = df.日期.str.split('/', expand = True)

#將 "時間" 的內容，切割、另建立 "時" 、 "分" 、 "秒" 3個不同的欄位
df[['時', '分', '秒']] = df.時間.str.split(':', expand = True)

#將所有的開頭為回文的字串，都取代成 "回文" 2字
df = df.replace({"主文/回文" : r"^回文.*$"}, {"主文/回文" : "回文"}, regex= True)

#### 2、按條件將資料做篩選

In [ ]:
#多重條件篩選
##檢視資料目前形式，df.head(顯示前幾列的資料)
ds = (df["網站"] != "Facebook公開社團") & (df["網站"] != "Facebook不公開社團") & (df["網站"] != "Facebook公眾訊息") & (df["網站"] != "Instagram")
df[ds].head(5)

In [ ]:
ds = (df["[維度]:麻辣*鍋"] == 1) | (df["[維度]:薑母鴨"] == 1) | (df["[維度]:羊肉爐"] == 1) | (df["[維度]:拌麵|麵線|烏龍麵|關廟麵|意麵|義大利麵|筆管麵|蕎麥麵|王子麵|雞蛋麵|刀削麵|寬麵|粄條|拉麵|韓國拉麵|韓國泡麵|泡麵|科學麵|蒸煮麵|冬粉|米粉"] == 1) | (df["[維度]:滷肉飯|魯肉飯|炒飯|油飯|白飯|粥"] == 1) | (df["[維度]:爆米花|提拉米蘇|蛋糕|滷味|冰淇淋|哈根達斯|雪糕|冰棒|湯圓"] == 1)
df[ds].head(3)

#### Excel輸出

In [ ]:
with pd.ExcelWriter(r"C:\Users\xphoe\OneDrive\文件\2021維度標記資料_python.xlsx", 
                    engine='xlsxwriter', 
                    options={'strings_to_urls': False}) as writer:
    df[ds].to_excel(writer, sheet_name='文章列表')